# Time-lapses with Google Earth Engine

**Set the environment**

Initialize Earth Engine account

In [1]:
import os
import ee
from ee import batch
import json
import requests
import folium
import urllib
import numpy as np
import pandas as pd
from IPython.display import display, Image
import matplotlib.pyplot as plt
from datetime import timedelta, date

ee.Initialize()

**Functions**

In [2]:
def display_image(image, region, Vizz = None):
    """
    Displays images in notebook
    """ 
    ## Visualization
    if Vizz:
        image = image.visualize(**Vizz)
        
    visual = Image(url=image.getThumbUrl({
                'region':region
                }))
    
    display(visual)

In [3]:
def download_image_png(image, path_out, scale, Vizz = None, region = None,):
    
    ## Visualization
    if Vizz:
        image = image.visualize(**Vizz)
        
    Vizparam = {'scale': scale, 'format': 'png', 'crs': 'EPSG:4326'}
    if region:
        Vizparam['region'] = region
    
    path = image.getThumbURL(Vizparam)
    
    if os.path.exists(path_out):
            os.remove(path_out)
    
    urllib.request.urlretrieve(path, path_out)

In [4]:
def maskS2clouds(image):
    """
    European Space Agency (ESA) clouds from 'QA60', i.e. Quality Assessment band at 60m
    parsed by Nick Clinton
    """
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = int(2**10)
    cirrusBitMask = int(2**11)

    # Both flags set to zero indicates clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(\
            qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

In [8]:
def Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale, bandNames = None):
    ## Define your collection
    collection = ee.ImageCollection(Collection_id)

    ## Filter 
    collection = collection.filterBounds(geom).filterDate(startDate,stopDate)\
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
        .map(maskS2clouds)

    ## Composite
    composite = collection.median()

    ## Choose the scale
    composite =  composite.reproject(crs='EPSG:4326', scale=scale)

    ## Select the bands
    if bandNames:
        composite = composite.select(bandNames)
    
    return composite

In [9]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    #If the cloud bit (5) is set and the cloud confidence (7) is high
    #or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3));
    #Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud.Not()).updateMask(mask2);

In [10]:
def Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale, bandNames = None):
    ## Define your collection
    collection = ee.ImageCollection(Collection_id)

    ## Filter 
    collection = collection.filterBounds(geom).filterDate(startDate,stopDate)\
            .map(cloudMaskL457)

    ## Composite
    composite = collection.median()

    ## Choose the scale
    composite =  composite.reproject(crs='EPSG:4326', scale=scale)

    ## Select the bands
    if bandNames:
        composite = composite.select(bandNames)
    
    return composite

## Sentinel 2

### Sentinel-2 MultiSpectral Instrument, Level-1C ([gee](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2))
**Dataset Availability**: 2015-06-23T00:00:00 - Present

**Wavebands**

|Band 	|Use 		|Wavelength (nm) |Resolution (m)|
|-------|-----------|----------------|--------------|
|B1 	|Aerosols 	|443 	|60|
|B2 	|Blue 		|490 	|10|
|B3 	|Green 		|560 	|10|
|B4 	|Red 		|665 	|10|
|B6 	|Red Edge 2 |740 	|20|
|B8 	|NIR        |835 	|10|
|B8a 	|Red Edge 4 |865 	|20|
|B9 	|Water vapor|940 	|60|
|B10 	|Cirrus 	|1375 	|60|
|B11 	|SWIR 1 	|1610 	|20|
|B12 	|SWIR 2 	|2190 	|20|
|QA60   |ESA Cloud  | n/a   |60|

In [27]:
# GEE Image Collection ID
Collection_id = 'COPERNICUS/S2'
# Area of Interest (AoI)
geom = ee.Geometry.Point(19.20356, -15.20109).buffer(4000)
region = geom.bounds().getInfo()['coordinates']
# Start and stop of time series
startDate = ee.Date('2016-01-01')
stopDate  = ee.Date('2016-06-30')
# Scale in meters
scale = 10
# Year 2016 monthly
period = np.array(pd.period_range('2016-01', '2017-12', freq='M').to_series().astype(str))

**Cloud Free Composite**

RGB

In [28]:
vis = {'min':0,'max':0.25, 'bands':['B4', 'B3', 'B2']}
image = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)
display_image(image, region, Vizz = vis)

**Export images as png**

In [ ]:
path = './Images/Sentinel2/RGB/RGB'
for i in range(len(period)-7):
    
    startDate = ee.Date(period[i])
    stopDate  = ee.Date(period[i+7])
    
    composite = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)

    # File name
    filename = '_'+'{:03d}'.format(i)+'.png'
    
    download_image_png(composite, path_out=path+filename, scale=scale, Vizz = vis, region = region)   

**Timelape**

In [ ]:
images = []
for i in range(len(period)-5):
    startDate = ee.Date(period[i])
    stopDate  = ee.Date(period[i+5])
    
    composite = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)
 
    # Visualization
    image = composite.visualize(**vis)
    
    images.append(image)
    
    collection = ee.ImageCollection(images)

In [ ]:
## make the data 8-bit.
def convertBit(image):
    return image.multiply(512).uint8()  

## call the conversion    
outputVideo = collection.map(convertBit)

print("about to build video")

## Export to video.
out = ee.batch.Export.video.toDrive(
    collection = outputVideo,
    folder = 'EarthPulse',
    description = 'Okavango_Delta_RGB', 
    dimensions = 720, 
    framesPerSecond = 2, 
    region = region,
    maxFrames = 10000
    )

## process the image
process = ee.batch.Task.start(out)

print("process sent to cloud")

NDVI = (RED-NIR)/(RED+NIR)

In [29]:
palette = ['blue', 'white', 'green']
vis = {'min': -0.5, 'max': 0.5, 'bands':'nd', 'palette': palette}

In [30]:
composite = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)
#Calculate NDVI
image_ndvi = composite.normalizedDifference(['B8','B4'])
display_image(image_ndvi, region, Vizz = vis)

**Export images as png**

In [ ]:
path = './Images/Sentinel2/NDVI/NDVI'
for i in range(len(period)-7):
    
    startDate = ee.Date(period[i])
    stopDate  = ee.Date(period[i+7])
    
    composite = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)
    
    # Calculate NDVI
    image = composite.normalizedDifference(['B8','B4'])
 
    # File name
    filename = '_'+'{:03d}'.format(i)+'.png'
    
    download_image_png(image, path_out=path+filename, scale=scale, Vizz = vis, region = region) 

NDWI = (GREEN-NIR)/(GREEN+NIR)

In [31]:
composite = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)
#Calculate NDWI
image_ndvi = composite.normalizedDifference(['B8','B3'])
display_image(image_ndvi, region, Vizz = vis)

**Export images as png**

In [ ]:
path = './Images/Sentinel2/NDWI/NDWI'
for i in range(len(period)-7):
    
    startDate = ee.Date(period[i])
    stopDate  = ee.Date(period[i+7])
    
    composite = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)
    
    # Calculate NDWI
    image = composite.normalizedDifference(['B8','B3'])
 
    # File name
    filename = '_'+'{:03d}'.format(i)+'.png'
    
    download_image_png(image, path_out=path+filename, scale=scale, Vizz = vis, region = region)   

MNDWI = (GREEN-SWIR)/(GREEN+SWIR)

In [32]:
composite = Cloud_Free_Composite_S2(Collection_id, startDate, stopDate, geom, scale = scale)
#Calculate MNDWI
image_ndvi = composite.normalizedDifference(['B11', 'B3'])
display_image(image_ndvi, region, Vizz = vis)

## Lansat 5

### USGS Landsat 5 Surface Reflectance Tier 1 ([gee](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C01_T1_SR))
**Dataset Availability**: January 1994 - May 2012

**Wavebands**

|Name	      |Resolution	|Wavelength	      |Description|
|-------------|-------------|-----------------|-----------|
|B1	          |30 meters	|0.45 - 0.52 µm	  |Blue                                                  |
|B2	          |30 meters	|0.52 - 0.60 µm	  |Green                                                 |
|B3	          |30 meters	|0.63 - 0.69 µm	  |Red                                                   |
|B4	          |30 meters	|0.77 - 0.90 µm	  |Near infrared                                         |
|B5	          |30 meters	|1.55 - 1.75 µm	  |Shortwave infrared 1                                  |
|B6	Kelvin	  |             |0.40 - 12.50 μm  |Band 6 brightness temperature                         |
|B7		 	  |             |2.08 - 2.35 μm	  |Band 7 (shortwave infrared 2) surface reflectance     |
|sr_atmos_opacity|          |                 |Atmospheric opacity                                   |
|sr_cloud_qa|               |                 |Cloud quality attributes                              |

In [33]:
# GEE Image Collection ID
Collection_id = 'LANDSAT/LT05/C01/T1_SR'
# Area of Interest (AoI)
#geom = ee.Geometry.Point(19.20356, -15.20109).buffer(10000)
#geom = ee.Geometry.Point(19.7368, -17.9489).buffer(12000)
geom = ee.Geometry.Point(17.6894, -14.6568).buffer(12000)
region = geom.bounds().getInfo()['coordinates']
# Start and stop of time series
startDate = ee.Date('1994-01-01')
stopDate  = ee.Date('1996-12-31')
# Scale in meters
scale = 30
# Period
#period = np.array(pd.period_range('1994-01-01', '1996-12-31', freq='M').to_series().astype(str))
period=np.array(pd.period_range('1994-01-01', '2010-12-31', freq='Y').to_series().astype(str))

**Cloud Free Composite**

RGB

In [34]:
vis = {'min':0,'max':3000, 'gamma':1.4, 'bands':['B3', 'B2', 'B1']}
image = Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale)
display_image(image, region, Vizz = vis)

In [ ]:
path = './Images/Lansat5/RGB/RGB'
step = 35
step = 2
for i in range(len(period)-step):
    
    startDate = ee.Date(period[i])
    stopDate  = ee.Date(period[i+step])
    
    composite = Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale)

    # File name
    filename = '_'+'{:03d}'.format(i)+'.png'
    
    download_image_png(composite, path_out=path+filename, scale=scale, Vizz = vis, region = region)  

## Lansat 7 

### USGS Landsat 7 Surface Reflectance Tier 1 ([gee](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_SR))
**Dataset Availability**: January 1999 - Present

**Wavebands**

|Name	      |Resolution	|Wavelength	      |Description|
|-------------|-------------|-----------------|-----------|
|B1	          |30 meters	|0.45 - 0.52 µm	  |Blue                                                  |
|B2	          |30 meters	|0.52 - 0.60 µm	  |Green                                                 |
|B3	          |30 meters	|0.63 - 0.69 µm	  |Red                                                   |
|B4	          |30 meters	|0.77 - 0.90 µm	  |Near infrared                                         |
|B5	          |30 meters	|1.55 - 1.75 µm	  |Shortwave infrared 1                                  |
|B6	Kelvin	  |             |0.40 - 12.50 μm  |Band 6 brightness temperature                         |
|B7		 	  |             |2.08 - 2.35 μm	  |Band 7 (shortwave infrared 2) surface reflectance     |
|sr_atmos_opacity|          |                 |Atmospheric opacity                                   |
|sr_cloud_qa|               |                 |Cloud quality attributes                              |

In [35]:
# GEE Image Collection ID
Collection_id = 'LANDSAT/LE07/C01/T1_SR'
# Area of Interest (AoI)
#geom = ee.Geometry.Point(19.20356, -15.20109).buffer(10000)
geom = ee.Geometry.Point(19.7368, -17.9489).buffer(12000)
#geom = ee.Geometry.Point(17.6894, -14.6568).buffer(12000)
region = geom.bounds().getInfo()['coordinates']
# Start and stop of time series
startDate = ee.Date('2015-01-01')
stopDate  = ee.Date('2017-12-31')
# Scale in meters
scale = 30
# Period
#period = np.array(pd.period_range('2015-01-01', '2017-12-31', freq='M').to_series().astype(str))
period=np.array(pd.period_range('2009-01-01', '2018-12-31', freq='Y').to_series().astype(str))

**Cloud Free Composite**

RGB

In [36]:
vis = {'min':0,'max':3000, 'gamma':1.4, 'bands':['B3', 'B2', 'B1']}
image = Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale)
display_image(image, region, Vizz = vis)

**Export images as png**

In [ ]:
path = './Images/Lansat7/RGB/RGB'
step = 35
step = 2
for i in range(len(period)-step):
    
    startDate = ee.Date(period[i])
    stopDate  = ee.Date(period[i+step])
    
    composite = Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale)

    # File name
    filename = '_'+'{:03d}'.format(i+15)+'.png'
    
    download_image_png(composite, path_out=path+filename, scale=scale, Vizz = vis, region = region)  

**Export images for different locations**

In [ ]:
# Path to save files
path = './Images/'
# Dictionaries with the different locations and collections
locations = {'Rundu': [19.7368, -17.9489], 'Menongue': [17.6894, -14.6568]}
#locations = {'Cuito': [19.20356, -15.20109], 'Rundu': [19.7368, -17.9489], 'Menongue': [17.6894, -14.6568]}
collections = {'Lansat_5': 'LANDSAT/LT05/C01/T1_SR', 'Lansat_7': 'LANDSAT/LE07/C01/T1_SR'}
# Visualization parametes
vis = {'min':0,'max':3000, 'gamma':1.4, 'bands':['B3', 'B2', 'B1']}
# Scale in meters
scale = 30

for location in locations.keys():
    print('Computing images in '+location)
    lng = locations[location][0]
    lat = locations[location][1]
    geom = ee.Geometry.Point(lng, lat).buffer(12000)
    region = geom.bounds().getInfo()['coordinates']
    
    n = 0
    step = 3
    for collection in collections.keys():
        Collection_id = collections[collection]
        if collection == 'Lansat_5':
            period = np.array(pd.period_range('1994-01-01', '2010-12-31', freq='Y').to_series().astype(str))
        else:
            period = np.array(pd.period_range('2008-01-01', '2018-12-31', freq='Y').to_series().astype(str))
            
        for i in range(len(period)-step):
            startDate = ee.Date(period[i])
            stopDate  = ee.Date(period[i+step])
    
            composite = Cloud_Free_Composite_L7_5(Collection_id, startDate, stopDate, geom, scale = scale)

            # File name
            filename = 'RGB_'+'{:03d}'.format(n)+'.png'
            
            download_image_png(composite, path_out=path+location+'/'+filename, scale=scale, Vizz = vis, region = region)    
            n+=1   

In [ ]:
step = 3
period = np.array(pd.period_range('1994-01-01', '2010-12-31', freq='Y').to_series().astype(str))
period = np.array(pd.period_range('2008-01-01', '2018-12-31', freq='Y').to_series().astype(str))
for i in range(len(period)-step):
    print(period[i])
    print(period[i+step])